In [1]:
from utils.util import mkdir,seed_all
from omegaconf import OmegaConf
from cprint import *
from datasets.shape_net import ShapeNet
import torch
from models.Transform2D import Transform2D
from tqdm.notebook import tqdm
import numpy as np
%load_ext autoreload
%autoreload 2

## Setup Expirement Folder

In [2]:
seed_all(111)
config = OmegaConf.load("./configs/global_configs.yaml")
cprint.ok(config)
description = "Testing Overfit" # Describe Experiment params here
logs_dir = config["logs_dir"]
mkdir(logs_dir)

experiment_dir = f"{logs_dir}/{config['name']}"
mkdir(experiment_dir)
mkdir(f"{experiment_dir}/checkpoints")
mkdir(f"{experiment_dir}/tb")
mkdir(f"{experiment_dir}/visuals")

{'logs_dir': 'logs', 'is_train': True, 'name': 'testingOverfit', 'device': 'cuda:0', 'batch_size': 16, 'n_epochs': 500, 'print_every': 300, 'validate_every': 300, 'model': {'lr': 0.0001}}
- logs directory found
- logs/testingOverfit directory found
- logs/testingOverfit/checkpoints directory found
- logs/testingOverfit/tb directory found
- logs/testingOverfit/visuals directory found


## Load Dataset & Dataloaders
This uses a random split for train/validation/test - might need to look into paper if they have a pre-defined split

In [3]:
dataset = ShapeNet(cat="airplane",is_overfit=False) #Change overfit param here & cat here
train_ds, valid_ds, test_ds = torch.utils.data.random_split(
    dataset, [0.7, 0.2, 0.1])

train_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )

validation_dataloader = torch.utils.data.DataLoader(
        train_ds,   # Datasets return data one sample at a time; Dataloaders use them and aggregate samples into batches
        batch_size=config['batch_size'],   # The size of batches is defined here
        shuffle=True,    # Shuffling the order of samples is useful during training to prevent that the network learns to depend on the order of the input data
        num_workers=4,   # Data is usually loaded in parallel by num_workers
        pin_memory=True,  # This is an implementation detail to speed up data uploading to the GPU
        # worker_init_fn=train_dataset.worker_init_fn  TODO: Uncomment this line if you are using shapenet_zip on Google Colab
    )


## Loading Model

In [4]:
model = Transform2D()
# Declare device
device = torch.device('cpu')
if torch.cuda.is_available() and config['device'].startswith('cuda'):
    device = torch.device(config['device'])
    cprint.ok('Using device:', config['device'])
else:
    cprint.warn('Using CPU')

model.to(device)


Using device: cuda:0


Transform2D(
  (encoder): Encoder(
    (conv1a): ConvBlock(
      (conv): Conv2d(4, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (conv1b): ConvBlock(
      (conv): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=1, dilation=1, ceil_mode=False)
    (conv2a): ConvBlock(
      (conv): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (conv2b): ConvBlock(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (relu): LeakyReLU(negative_slope=0.01)
    )
    (res2): ResidualBlock(
      (conv1): ConvBlock(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): LeakyReLU(negative_slope=0.01)
      )
      (conv2): ConvBlock(
        (conv): Conv2d(

## Training

Checklist:
- Add tensorboard and make sure it logs to the `${experiment_dir}/tb` folder
- Visualize some reconstructions on validation set and make sure it logs to the `${expirement_dir}/visuals` folders

In [5]:
train_loss_running = 0.
best_loss_val = np.inf
model.train()

for epoch in tqdm(range(config['n_epochs'])):
     for batch_idx, batch in tqdm(enumerate(train_dataloader)):
          ShapeNet.move_batch_to_device(batch, device)
          model.step(batch)
          train_loss_running += model.get_current_errors()
          iteration = epoch * len(train_dataloader) + batch_idx
          if iteration % config['print_every'] == (config['print_every'] - 1):
            cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] train_loss: {train_loss_running / config["print_every_n"]:.6f}')
            train_loss_running = 0.


     if iteration % config['validate_every'] == (config['validate_every'] - 1 or True):
          model.eval()
          loss_val = 0.
          for batch_val in validation_dataloader:
               ShapeNet.move_batch_to_device(batch_val, device)
               with torch.no_grad():
                   model.set_input(batch_val)
                   output = model.inference()
                   loss_val += model.get_current_errors()
          
          if loss_val < best_loss_val:
                    model.save()
                    best_loss_val = loss_val

          cprint.warn(f'[{epoch:03d}/{batch_idx:05d}] val_loss: {loss_val:.6f} | best_loss_val: {best_loss_val:.6f}')
     model.update_lr()
         

  0%|          | 0/500 [00:00<?, ?it/s]

0it [00:00, ?it/s]

ValueError: too many values to unpack (expected 5)

## Training on  single instance for checking

In [6]:

train_loss_running = 0.
best_loss_val = np.inf
model.train()
data = dataset[0]
#data["images"] = data["images"].squeeze(0)
data["images"] = torch.tensor(data["images"]).cuda().float()
data["voxels"] = torch.tensor(data["voxels"]).cuda().float()
for epoch in tqdm(range(400)):
    model.step(data)
    loss = model.get_loss()
    cprint.ok(f'Model Loss: {loss["loss"]}')


  0%|          | 0/400 [00:00<?, ?it/s]

Model Loss: 0.6859337687492371
Model Loss: 0.6842008233070374
Model Loss: 0.6824297904968262
Model Loss: 0.6806228756904602
Model Loss: 0.6787914037704468
Model Loss: 0.67694091796875
Model Loss: 0.6750166416168213
Model Loss: 0.673008143901825
Model Loss: 0.6709796190261841
Model Loss: 0.6688670516014099
Model Loss: 0.6666658520698547
Model Loss: 0.6643546223640442
Model Loss: 0.6619202494621277
Model Loss: 0.6593294143676758
Model Loss: 0.6565394997596741
Model Loss: 0.6535157561302185
Model Loss: 0.6502358913421631
Model Loss: 0.6466203331947327
Model Loss: 0.6425511837005615
Model Loss: 0.6378945708274841
Model Loss: 0.6325564384460449
Model Loss: 0.6263343095779419
Model Loss: 0.6189380288124084
Model Loss: 0.6100137233734131
Model Loss: 0.5991069674491882
Model Loss: 0.5855186581611633
Model Loss: 0.5686181783676147
Model Loss: 0.5478572845458984
Model Loss: 0.523469865322113
Model Loss: 0.4974367916584015
Model Loss: 0.4748135209083557
Model Loss: 0.46174150705337524
Model Loss:

Model Loss: 0.07230307161808014
Model Loss: 0.0719820037484169
Model Loss: 0.07173015177249908
Model Loss: 0.07144300639629364
Model Loss: 0.07098915427923203
Model Loss: 0.07071730494499207
Model Loss: 0.07056374102830887
Model Loss: 0.07023796439170837
Model Loss: 0.06998644769191742
Model Loss: 0.06989552080631256
Model Loss: 0.06948399543762207
Model Loss: 0.06933550536632538
Model Loss: 0.0694330632686615
Model Loss: 0.07056134939193726
Model Loss: 0.07713457196950912
Model Loss: 0.07260799407958984
Model Loss: 0.07202037423849106
Model Loss: 0.0701969638466835
Model Loss: 0.07087142765522003
Model Loss: 0.07188121974468231
Model Loss: 0.07110505551099777
Model Loss: 0.06987117230892181
Model Loss: 0.06935545057058334
Model Loss: 0.07034340500831604
Model Loss: 0.0702897235751152
Model Loss: 0.0693199634552002
Model Loss: 0.06905371695756912
Model Loss: 0.06917251646518707
Model Loss: 0.06908652186393738
Model Loss: 0.06883978843688965
Model Loss: 0.06870115548372269
Model Loss: 0

In [6]:
pred = model.x

In [10]:
from utils.visualizations import visualize_occupancy,visualize_images


In [12]:
visualize_occupancy(data["voxels"][0].cpu(),flip_axes=True)

Output()

In [7]:
x = model(data)

In [8]:
pred = x[0]
pred[pred<0.4] =0 
pred[pred>0.4] =1
# pred.shape
# tgt = data["voxels"]
# tgt[tgt!=0].shape

pred[pred!=0].shape

torch.Size([349])

In [11]:
visualize_occupancy(pred.detach().cpu().numpy(),flip_axes=True)

Output()